In [10]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import sys, os
import time
import multiprocessing
ncpus = multiprocessing.cpu_count()
#from BurnCube import BurnCube #including burn mapping main functions
#bc = BurnCube()

from datacube_stats.statistics import GeoMedian

import datacube
from datacube.storage import masking
from datacube.storage.storage import write_dataset_to_netcdf
from datacube.helpers import write_geotiff
dc = datacube.Datacube(app='dc_2nd_order_stata')

sys.path.append(os.path.abspath('/g/data/r78/DPIPWE_lm/dea-notebooks/10_Scripts'))
import DEAPlotting
import DEADataHandling
import BandIndices

sys.path.append(os.path.abspath('/home/554/lm4502/datacube-2nd-order-stats/model'))
#import DEAPlotting
#import DEADataHandling

'''
outputdir = '/g/data/r78/DPIPWE_lm/test_burn_mapping/output_data'
if not os.path.exists(outputdir):
    print("output directory doesn't exist")
    exit()
'''

'\noutputdir = \'/g/data/r78/DPIPWE_lm/test_burn_mapping/output_data\'\nif not os.path.exists(outputdir):\n    print("output directory doesn\'t exist")\n    exit()\n'

In [15]:
query_PL = {
                'time': ('2018-01-25'),
                'x': (1129000, 1089000),
                'y': (-4590000, -4558000),
                'crs': 'EPSG:3577'
                }

In [16]:
#dc_data2 = DEADataHandling.load_clearlandsat(dc=dc, product='fc', query=query_PL, ls7_slc_off=True, masked_prop=0)
dc_data2 = dc.load(product='ls8_nbart_albers', group_by = 'solar_day', **query_PL)

In [17]:
dc_data2_ndvi = BandIndices.calculate_indices(dc_data2, 'NDVI')

The formula we are using is (nir - red)/(nir + red)


In [18]:
dc_data2_ndvi

<xarray.DataArray (time: 1, y: 1280, x: 1600)>
array([[[ 0.659514,  0.682012, ...,  0.862408,  0.867044],
        [ 0.693112,  0.66802 , ...,  0.864823,  0.870144],
        ...,
        [-0.05618 , -0.105263, ...,  0.851693,  0.884568],
        [-0.094972, -0.070423, ...,  0.830882,  0.889605]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-25T00:04:12
  * y        (y) float64 -4.558e+06 -4.558e+06 ... -4.59e+06 -4.59e+06
  * x        (x) float64 1.089e+06 1.089e+06 1.089e+06 ... 1.129e+06 1.129e+06

In [21]:
diff_ndvi = dc_data2_ndvi - dc_data_ndvi
diff_ndvi

<xarray.DataArray (time: 0, y: 1280, x: 1600)>
array([], shape=(0, 1280, 1600), dtype=float64)
Coordinates:
  * time     (time) datetime64[ns] 
  * y        (y) float64 -4.558e+06 -4.558e+06 ... -4.59e+06 -4.59e+06
  * x        (x) float64 1.089e+06 1.089e+06 1.089e+06 ... 1.129e+06 1.129e+06

In [25]:
ds_diff_ndvi = diff_ndvi.to_dataset(name='diff_ndvi')
ds_diff_ndvi

<xarray.Dataset>
Dimensions:    (time: 0, x: 1600, y: 1280)
Coordinates:
  * time       (time) datetime64[ns] 
  * y          (y) float64 -4.558e+06 -4.558e+06 ... -4.59e+06 -4.59e+06
  * x          (x) float64 1.089e+06 1.089e+06 1.089e+06 ... 1.129e+06 1.129e+06
Data variables:
    diff_ndvi  (time, y, x) float64 
Attributes:
    crs:      EPSG:3577

In [40]:
dc_data2.geographic

AttributeError: 'Dataset' object has no attribute 'geographic'

In [36]:
ds_diff_ndvi.attrs['crs'] = 'EPSG:3577'
ds_diff_ndvi.attrs['affine'] = dc_data2.affine

In [27]:
#DEAPlotting.three_band_image(ds_diff_ndvi, contrast_enhance=True);

In [28]:
#DEAPlotting.three_band_image(dc_data, bands = ['BS', 'PV', 'NPV'], contrast_enhance=True);
#DEAPlotting.three_band_image(dc_data2, bands = ['BS', 'PV', 'NPV'], contrast_enhance=True);

In [29]:
#ds_comb = xr.concat([dc_data, dc_data2], dim='time')
#ds_comb

In [30]:
#dc_diff = ds_comb.diff('time')
#dc_diff

In [31]:
#dc_diff

In [32]:
#DEAPlotting.three_band_image(dc_diff, bands = ['BS', 'PV', 'NPV'], contrast_enhance=True);

In [41]:
#DEADataHandling.dataset_to_geotiff('PIPELINE_ls8_fc_20180125_20160221.tif', dc_data.isel(time=0))
#datacube.helpers.write_geotiff(filename='PIPELINE_ls8_ndvi_diff.tif', dataset=ds_diff_ndvi)
#dc_data
datacube.storage.storage.write_dataset_to_netcdf(ds_diff_ndvi, 'PIPELINE_ls8_ndvi_diff.nc')

TypeError: expected str, bytes or os.PathLike object, not Dataset